In [1]:
%pip install -U langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.1.1
    Uninstalling langchain-core-1.1.1:
      Successfully uninstalled langchain-core-1.1.1
  Attempting uninstall: langchain
    Found existing installation: langchain 1.1.2
    Uninstalling langchain-1.1.2:
      Successfully uninstalled langchain-1.1.2


In [7]:
from google.colab import userdata
API_KEY = userdata.get('SILICONFLOW_API_KEY')

**本课重点：**

1. **Middleware 核心概念** - 理解 Agent 执行流程和钩子系统
2. **装饰器式 Middleware** - 使用 `@before_model`、`@after_model`、`@wrap_model_call` 等
3. **类式 Middleware** - 继承 `AgentMiddleware` 创建复杂中间件
4. **内置 Middleware** - 使用官方提供的 12+ 个预置中间件
5. **高级模式** - 动态提示、模型降级、人工审核等
6. **结构化输出** - 与 Pydantic 结合使用
7. **实战项目** - 构建生产级智能助手



### 什么是 Middleware？

Middleware 是在 Agent 执行流程中插入的钩子（hooks），用于：

- ✅ **日志与分析** - 追踪行为、调试、性能监控
- ✅ **转换** - 修改提示词、工具选择、输出格式
- ✅ **容错** - 重试、降级、早期终止
- ✅ **安全** - 限流、守护规则、PII 检测



Agent 执行流程

```
┌─────────────────────────────────────────────────────┐
│                   Agent 循环                         │
├─────────────────────────────────────────────────────┤
│                                                     │
│  1. before_model ──→ [准备阶段]                     │
│         ↓                                           │
│  2. modify_model_request ──→ [修改请求]             │
│         ↓                                           │
│  3. [调用模型] ← wrap_model_call                    │
│         ↓                                           │
│  4. after_model ──→ [处理响应]                      │
│         ↓                                           │
│  5. [工具执行] ← wrap_tool_call                     │
│         ↓                                           │
│  6. [结束或继续循环]                                │
│                                                     │
└─────────────────────────────────────────────────────┘
```

### 核心钩子（Hooks）一览

| 钩子                    | 时机         | 用途             | 可否修改状态 | 可否跳转 |
| ----------------------- | ------------ | ---------------- | ------------ | -------- |
| `@before_agent`         | Agent 开始前 | 初始化、权限检查 | ✅            | ✅        |
| `@before_model`         | 模型调用前   | 日志、状态更新   | ✅            | ✅        |
| `@modify_model_request` | 修改模型请求 | 改提示词、工具   | ❌            | ❌        |
| `@wrap_model_call`      | 包裹模型调用 | 重试、缓存       | ❌            | ❌        |
| `@after_model`          | 模型响应后   | 日志、验证       | ✅            | ✅        |
| `@wrap_tool_call`       | 包裹工具调用 | 监控、重试       | ❌            | ❌        |
| `@dynamic_prompt`       | 生成系统提示 | 动态上下文       | ❌            | ❌        |
| `@after_agent`          | Agent 结束后 | 清理、统计       | ✅            | ❌        |

In [4]:

"""
最简单的 Middleware 示例
"""

from langchain.agents import create_agent
from langchain.agents.middleware import before_model, after_model, AgentState
from langchain.chat_models import init_chat_model
from langgraph.runtime import Runtime
from typing import Any
from dotenv import load_dotenv


# 1. 定义 before_model 钩子
@before_model
def log_before(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """在每次调用模型前打印日志"""
    print(f"📝 before_model准备调用模型，当前消息数：{len(state['messages'])}")
    return None  # 不修改状态

# 2. 定义 after_model 钩子
@after_model
def log_after(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """在每次模型响应后打印日志"""
    last_message = state['messages'][-1]
    print(f"✅ after_model模型响应：{last_message.content[:50]}...")
    return None

# 3. 创建模型
model = init_chat_model(
    "deepseek-chat", # Changed from "deep-seek" to "deepseek-chat"
    model_provider="openai",
    base_url="https://api.deepseek.com",
    api_key= API_KEY
)

# 4. 创建 Agent（添加 middleware）
agent = create_agent(
    model=model,
    tools=[],
    middleware=[log_before, log_after]  # 👈 添加中间件
)

# 5. 测试
result = agent.invoke({"messages": [{"role": "user", "content": "你好！"}]})
print(result)
print(f"\n最终结果：{result['messages'][-1].content}")

📝 before_model准备调用模型，当前消息数：1
✅ after_model模型响应：你好！很高兴见到你！😊 我是DeepSeek，由深度求索公司创造的AI助手。无论你有什么问题、需要什...
{'messages': [HumanMessage(content='你好！', additional_kwargs={}, response_metadata={}, id='86ab7aa8-63fb-4882-ac2f-c7eaf91b3b56'), AIMessage(content='你好！很高兴见到你！😊 我是DeepSeek，由深度求索公司创造的AI助手。无论你有什么问题、需要什么帮助，或者只是想聊聊天，我都很乐意为你提供支持！\n\n我可以帮你解答各种问题，协助处理文档，进行创作和分析等等。而且我是完全免费的哦！有什么我可以为你做的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 6, 'total_tokens': 76, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 6}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '14d0f9e8-e680-405d-9338-f0e79e6adecd', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b083d-b3ab-7983-bfcf-8388df135492-0', usage_metadata={'input_tokens': 6, 'output_tokens':

### `@before_model` 和 `@after_model`

**签名：**

```
def before_model(
    state: AgentState,
    runtime: Runtime
) -> dict[str, Any] | None:
    """
    返回值：
    - None: 不修改状态
    - dict: 更新状态，可包含 "jump_to" 键跳转节点
    """
    
```




In [5]:
# **示例 1：消息数量限制**

from langchain.agents.middleware import before_model, AgentState
from langchain.messages import AIMessage
from langgraph.runtime import Runtime
from typing import Any

@before_model(can_jump_to=["end"])  # 👈 声明可以跳转到 "end"
def check_message_limit(
    state: AgentState,
    runtime: Runtime
) -> dict[str, Any] | None:
    """限制对话轮次，超过 50 轮自动结束"""
    if len(state["messages"]) >= 50:
        print("⚠️  对话已达上限，自动结束")
        return {
            "messages": [AIMessage(content="对话已达上限，感谢使用！")],
            "jump_to": "end"  # 👈 跳转到结束节点
        }
    return None


# **示例 2：统计 Token 使用**
from langchain.agents.middleware import after_model, AgentState
from langgraph.runtime import Runtime
from typing import Any

class TokenCounter:
    """Token 计数器"""
    def __init__(self):
        self.total_tokens = 0

    def create_hook(self):
        """创建钩子闭包"""
        @after_model
        def count_tokens(
            state: AgentState,
            runtime: Runtime
        ) -> dict[str, Any] | None:
            # 从模型响应中提取 token 使用情况
            last_message = state["messages"][-1]
            if hasattr(last_message, "response_metadata"):
                usage = last_message.response_metadata.get("usage", {})
                tokens = usage.get("total_tokens", 0)
                self.total_tokens += tokens
                print(f"📊 本次使用 {tokens} tokens，累计 {self.total_tokens}")
            return None

        return count_tokens

# 使用
counter = TokenCounter()
agent = create_agent(
    model=model,
    middleware=[counter.create_hook()]
)

# `@wrap_model_call` - 包裹模型调用
用于拦截、重试、缓存模型调用。



```
def wrap_model_call(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """
    request: 包含 model, messages, system_message, tools, state
    handler: 执行实际模型调用的函数
    返回：ModelResponse
    """
```





**示例 1：重试逻辑**

In [ ]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable
import time

@wrap_model_call
def retry_model(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """自动重试失败的模型调用"""
    max_retries = 3

    for attempt in range(max_retries):
        try:
            print(f"🔄 尝试调用模型（第 {attempt + 1}/{max_retries} 次）")
            return handler(request)
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"❌ 所有重试失败：{e}")
                raise

            # 指数退避
            wait_time = 2 ** attempt
            print(f"⚠️  调用失败：{e}，{wait_time} 秒后重试")
            time.sleep(wait_time)

**示例 2：响应缓存**

In [ ]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable
import hashlib
import json

class ModelCache:
    """模型响应缓存"""
    def __init__(self):
        self.cache = {}

    def create_hook(self):
        @wrap_model_call
        def cache_model(
            request: ModelRequest,
            handler: Callable[[ModelRequest], ModelResponse]
        ) -> ModelResponse:
            # 生成缓存键
            cache_key = hashlib.md5(
                json.dumps({
                    "messages": [str(m) for m in request.messages],
                    "system": str(request.system_message)
                }).encode()
            ).hexdigest()

            # 检查缓存
            if cache_key in self.cache:
                print("💾 缓存命中！")
                return self.cache[cache_key]

            # 调用模型
            print("🔍 缓存未命中，调用模型")
            response = handler(request)

            # 存入缓存
            self.cache[cache_key] = response
            return response

        return cache_model

# 使用
cache = ModelCache()
agent = create_agent(
    model=model,
    middleware=[cache.create_hook()]
)

**示例 3：修改系统提示**

In [ ]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain_core.messages import SystemMessage
from typing import Callable

@wrap_model_call
def add_context(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """动态添加上下文信息到系统提示"""

    # 获取当前时间
    from datetime import datetime
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # 构建新的系统消息
    original_content = request.system_message.content if request.system_message else ""
    new_content = f"""{original_content}

      当前时间：{current_time}
      用户位置：中国
      语言偏好：中文
      """

    # 创建新的系统消息
    new_system_message = SystemMessage(content=new_content)

    # 使用 override 方法修改请求
    modified_request = request.override(system_message=new_system_message)

    return handler(modified_request)

# `@wrap_tool_call` - 包裹工具调用
用于监控、重试、修改工具执行。

In [ ]:
from langchain.agents.middleware import wrap_tool_call
from langchain.tools.tool_node import ToolCallRequest
from langchain_core.messages import ToolMessage
from langgraph.types import Command
from typing import Callable
import time

@wrap_tool_call
def monitor_tool(
    request: ToolCallRequest,
    handler: Callable[[ToolCallRequest], ToolMessage | Command]
) -> ToolMessage | Command:
    """监控工具执行时间和状态"""

    tool_name = request.tool_call["name"]
    tool_args = request.tool_call.get("args", {})

    print(f"🔧 开始执行工具：{tool_name}")
    print(f"   参数：{tool_args}")

    start_time = time.time()
    try:
        result = handler(request)
        elapsed = time.time() - start_time
        print(f"✅ 工具执行成功，耗时：{elapsed:.2f}秒")
        return result
    except Exception as e:
        elapsed = time.time() - start_time
        print(f"❌ 工具执行失败：{e}，耗时：{elapsed:.2f}秒")
        raise

# `@dynamic_prompt` - 动态系统提示

根据上下文动态生成系统提示词。

In [ ]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """根据用户角色生成不同的系统提示"""

    # 从 runtime context 获取用户角色
    user_role = request.runtime.context.get("user_role", "user")

    base_prompt = "你是一个专业的AI助手。"

    if user_role == "expert":
        return f"{base_prompt}\n请提供详细的技术解释，使用专业术语。"
    elif user_role == "beginner":
        return f"{base_prompt}\n请用简单易懂的语言解释，避免专业术语。"
    elif user_role == "child":
        return f"{base_prompt}\n请用儿童能理解的方式解释，使用比喻和故事。"
    else:
        return base_prompt

# 使用时传入 context
agent = create_agent(
    model=model,
    middleware=[user_role_prompt],
    tools=[]
)

# 调用时指定用户角色
result = agent.invoke(
    {"messages": [{"role": "user", "content": "什么是人工智能？"}]},
    config={"configurable": {"user_role": "child"}}
)

# 创建(类式)自定义 Middleware

当需要多个钩子或有状态时，使用类继承 `AgentMiddleware`。

In [ ]:
from langchain.agents.middleware import AgentMiddleware, AgentState, ModelRequest, ModelResponse
from langgraph.runtime import Runtime
from typing import Any, Callable

class CustomMiddleware(AgentMiddleware):
    """自定义中间件基础模板"""

    def __init__(self, config: dict = None):
        """初始化配置"""
        self.config = config or {}

    def before_agent(
        self,
        state: AgentState,
        runtime: Runtime
    ) -> dict[str, Any] | None:
        """Agent 开始前执行"""
        return None

    def before_model(
        self,
        state: AgentState,
        runtime: Runtime
    ) -> dict[str, Any] | None:
        """模型调用前执行"""
        return None

    def modify_model_request(
        self,
        request: ModelRequest
    ) -> ModelRequest:
        """修改模型请求"""
        return request

    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse]
    ) -> ModelResponse:
        """包裹模型调用"""
        return handler(request)

    def after_model(
        self,
        state: AgentState,
        runtime: Runtime
    ) -> dict[str, Any] | None:
        """模型响应后执行"""
        return None

    def wrap_tool_call(
        self,
        request: ToolCallRequest,
        handler: Callable[[ToolCallRequest], ToolMessage | Command]
    ) -> ToolMessage | Command:
        """包裹工具调用"""
        return handler(request)

    def after_agent(
        self,
        state: AgentState,
        runtime: Runtime
    ) -> dict[str, Any] | None:
        """Agent 结束后执行"""
        return None

**实战示例：完整的日志和性能监控中间件**

In [ ]:
from langchain.agents.middleware import AgentMiddleware, AgentState, ModelRequest, ModelResponse
from langchain.tools.tool_node import ToolCallRequest
from langchain_core.messages import ToolMessage
from langgraph.types import Command
from langgraph.runtime import Runtime
from typing import Any, Callable
import time
import json
from datetime import datetime

class PerformanceMonitorMiddleware(AgentMiddleware):
    """性能监控中间件"""

    def __init__(self, log_file: str = "agent_performance.jsonl"):
        self.log_file = log_file
        self.session_start = None
        self.model_call_count = 0
        self.tool_call_count = 0
        self.total_model_time = 0.0
        self.total_tool_time = 0.0

    def before_agent(
        self,
        state: AgentState,
        runtime: Runtime
    ) -> dict[str, Any] | None:
        """记录会话开始"""
        self.session_start = time.time()
        print(f"🚀 Agent 会话开始：{datetime.now()}")
        return None

    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse]
    ) -> ModelResponse:
        """监控模型调用"""
        start = time.time()
        self.model_call_count += 1

        try:
            response = handler(request)
            elapsed = time.time() - start
            self.total_model_time += elapsed

            print(f"📊 模型调用 #{self.model_call_count}：{elapsed:.2f}秒")
            return response
        except Exception as e:
            elapsed = time.time() - start
            print(f"❌ 模型调用失败：{e}（耗时 {elapsed:.2f}秒）")
            raise

    def wrap_tool_call(
        self,
        request: ToolCallRequest,
        handler: Callable[[ToolCallRequest], ToolMessage | Command]
    ) -> ToolMessage | Command:
        """监控工具调用"""
        start = time.time()
        self.tool_call_count += 1
        tool_name = request.tool_call["name"]

        try:
            result = handler(request)
            elapsed = time.time() - start
            self.total_tool_time += elapsed

            print(f"🔧 工具调用 #{self.tool_call_count} ({tool_name})：{elapsed:.2f}秒")
            return result
        except Exception as e:
            elapsed = time.time() - start
            print(f"❌ 工具调用失败 ({tool_name})：{e}（耗时 {elapsed:.2f}秒）")
            raise

    def after_agent(
        self,
        state: AgentState,
        runtime: Runtime
    ) -> dict[str, Any] | None:
        """输出性能统计"""
        total_time = time.time() - self.session_start

        stats = {
            "timestamp": datetime.now().isoformat(),
            "total_time": round(total_time, 2),
            "model_calls": self.model_call_count,
            "tool_calls": self.tool_call_count,
            "avg_model_time": round(self.total_model_time / max(self.model_call_count, 1), 2),
            "avg_tool_time": round(self.total_tool_time / max(self.tool_call_count, 1), 2),
        }

        print("\n" + "="*60)
        print("📊 性能统计")
        print("="*60)
        print(f"总耗时：{stats['total_time']}秒")
        print(f"模型调用：{stats['model_calls']} 次，平均 {stats['avg_model_time']}秒")
        print(f"工具调用：{stats['tool_calls']} 次，平均 {stats['avg_tool_time']}秒")
        print("="*60 + "\n")

        # 写入日志文件
        with open(self.log_file, 'a', encoding='utf-8') as f:
            f.write(json.dumps(stats, ensure_ascii=False) + "\n")

        return None

#   LangChain 官方内置 Middleware

LangChain 1.0 提供了 **12+ 个预置中间件**，开箱即用。

#### 1、`SummarizationMiddleware` - 对话历史总结

当对话历史超过 token 限制时，自动总结历史消息。

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model="openai:gpt-4o",
    tools=[],
    middleware=[
        SummarizationMiddleware(
            model="openai:gpt-4o-mini",  # 用更便宜的模型总结
            max_tokens_before_summary=400,  # 超过 400 tokens 触发总结
            messages_to_keep=5  # 保留最近 5 条消息
        )
    ]
)

#### 2、 `HumanInTheLoopMiddleware` - 人工审核
暂停 Agent 执行，让用户审核、编辑或拒绝工具调用。


In [6]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware

agent = create_agent(
    model="openai:gpt-4o",
    tools=[web_search, send_email],  # 敏感操作
    middleware=[
        HumanInTheLoopMiddleware(
            # 需要人工审核的工具
            tools_requiring_approval=["send_email"],
            # 或者所有工具都需要审核
            # require_approval_for_all_tools=True
        )
    ]
)

# 执行时会暂停等待人工确认
result = agent.invoke({"messages": [{"role": "user", "content": "发邮件给老板"}]})

NameError: name 'web_search' is not defined

#### 3、`ModelCallLimitMiddleware` - 模型调用限制
防止无限循环或滥用。

In [ ]:
from langchain.agents.middleware import ModelCallLimitMiddleware

limiter = ModelCallLimitMiddleware(
    thread_limit=20,  # 每个线程最多 20 次模型调用
    run_limit=10,     # 每次运行最多 10 次
    exit_behavior="continue"  # 达到限制后继续执行（不抛异常）
)

agent = create_agent(
    model="openai:gpt-4o",
    middleware=[limiter]
)

#### 4、`ToolCallLimitMiddleware` - 工具调用限制

In [ ]:
from langchain.agents.middleware import ToolCallLimitMiddleware

tool_limiter = ToolCallLimitMiddleware(
    thread_limit=50,  # 每个线程最多 50 次工具调用
    run_limit=20      # 每次运行最多 20 次
)

agent = create_agent(
    model="openai:gpt-4o",
    tools=[calculator, web_search],
    middleware=[tool_limiter]
)

#### 5、`PIIMiddleware` - 敏感信息保护

检测并处理个人身份信息（PII）。

| 策略 (strategy) | 含义 / 行为                                                                                                                                                                                                                 |
| ------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **`block`**   | 如果检测到 PII，就**抛出错误**（exception / `PIIDetectionError`），**阻止执行** — 不允许继续。适合对隐私要求极高、绝不允许泄露任何敏感信息的场景。([LangChain Reference][1])                                                                                              |
| **`redact`**  | 将检测到的 PII **替换为占位符** (placeholder)，通常是类似 `[REDACTED_EMAIL]`、`[REDACTED_CREDIT_CARD]` 这样的标签 — 即完全“擦除/隐藏”真实内容。适合日志清洗、合规需求、公开输出时隐藏敏感内容。([LangChain Reference][1])                                                          |
| **`mask`**    | 对 PII 进行**部分遮蔽(masking)**，比如信用卡号可能变成 `****-****-****-1234`（只保留最后几位/部分可见），邮箱可能保留域名部分 + 隐藏用户名的一部分等 — 既隐藏大部分敏感信息，又保留了一点“可辨识性”（比如账号后四位、域名等），适合用户服务界面 / 前端显示 / 需要部分可识别但不泄露完整敏感内容的场景。([LangChain Reference][1])               |
| **`hash`**    | 将检测到的 PII 转换成一个**确定性 hash 值** (deterministic hash)，比如 `<email_hash:a1b2c3d4>` 这样的标识 — 对于原始用户来说是真匿名／不可逆的，但对系统来说可以“区分/去重/追踪”（比如统计某个邮箱是否出现过多次、做行为分析、日志对比等）。适合 analytics、调试 (debug)、统计分析、匿名追踪等场景。([LangChain Reference][1]) |

[1]: https://reference.langchain.com/python/langchain/middleware/?utm_source=chatgpt.com "Middleware | LangChain Reference"


In [ ]:
from langchain.agents.middleware import PIIMiddleware

pii_protector = PIIMiddleware(
    detection_model="openai:gpt-4o-mini",
    strategy="redact",  # 策略：block, redact, mask, hash
    pii_types=["email", "phone", "ssn", "credit_card"]
)

agent = create_agent(
    model="openai:gpt-4o",
    middleware=[pii_protector],
    tools=[]
)

# 输入包含敏感信息会自动处理
result = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "我的邮箱是 user@example.com，电话 13800138000"
    }]
})

#### 6、`ModelFallbackMiddleware` - 模型降级

主模型失败时自动切换备用模型。

In [ ]:
from langchain.agents.middleware import ModelFallbackMiddleware
from langchain.chat_models import init_chat_model

# 定义主模型和备用模型
primary_model = init_chat_model("openai:gpt-4o")
fallback_models = [
    init_chat_model("openai:gpt-4o-mini"),
    init_chat_model("anthropic:claude-3-haiku")
]

fallback = ModelFallbackMiddleware(
    fallback_models=fallback_models
)

agent = create_agent(
    model=primary_model,
    middleware=[fallback],
    tools=[]
)

#### 7、`ToolRetryMiddleware` - 工具重试

自动重试失败的工具调用。

第 1 次重试（retry_number=1）: 等待 ~ 1.0 * (2.0 ** 1) = 2.0 秒

第 2 次重试（retry_number=2）: 等待 ~ 1.0 * (2.0 ** 2) = 4.0 秒

第 3 次重试（retry_number=3）: 等待 ~ 1.0 * (2.0 ** 3) = 8.0 秒

也就是说，等待时间以指数方式增长 —— 每失败一次，下次再试之前等待更长时间。

如果你把 backoff_factor = 0，就意味着不使用指数增长，重试之间始终用固定的 initial_delay。

In [ ]:
from langchain.agents.middleware import ToolRetryMiddleware

retry = ToolRetryMiddleware(
    max_retries=3,
    backoff_factor=2.0,  # 指数退避因子
    retry_on_exceptions=[ConnectionError, TimeoutError]
)

agent = create_agent(
    model="openai:gpt-4o",
    tools=[web_search, api_call],
    middleware=[retry]
)

#### 8、`FilesystemFileSearchMiddleware` - 文件系统搜索

为 Agent 添加文件搜索能力（Glob 和 Grep）。

In [ ]:
from langchain.agents.middleware import FilesystemFileSearchMiddleware

file_search = FilesystemFileSearchMiddleware(
    root_path="/workspace/project",  # 搜索根目录
    allowed_extensions=[".py", ".js", ".md"],  # 允许的文件类型
    max_results=50
)

agent = create_agent(
    model="openai:gpt-4o",
    tools=[],  # 自动添加 Glob 和 Grep 工具
    middleware=[file_search]
)

# Agent 现在可以搜索文件
result = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "帮我找到所有包含 'API' 的 Python 文件"
    }]
})

#### 9、`LLMToolSelectorMiddleware` - 智能工具筛选

当工具太多时，用 LLM 筛选最相关的工具。

In [ ]:
from langchain.agents.middleware import LLMToolSelectorMiddleware

tool_selector = LLMToolSelectorMiddleware(
    model="openai:gpt-4o-mini",
    max_tools=5  # 最多选择 5 个工具
)

agent = create_agent(
    model="openai:gpt-4o",
    tools=[...100个工具...],  # 很多工具
    middleware=[tool_selector]
)

### 组合多个 Middleware

Middleware 可以叠加使用，**执行顺序**：



```
进入：[MW1 before] → [MW2 before] → [MW3 before] → 模型调用
返回：[MW3 after] → [MW2 after] → [MW1 after]

```



In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import (
    SummarizationMiddleware,
    ModelCallLimitMiddleware,
    PIIMiddleware,
    ToolRetryMiddleware
)

agent = create_agent(
    model="openai:gpt-4o",
    tools=[web_search, calculator],
    middleware=[
        PIIMiddleware(strategy="redact"),           # 1. 最先检查 PII
        ModelCallLimitMiddleware(run_limit=10),     # 2. 限制调用次数
        SummarizationMiddleware(max_tokens_before_summary=500),  # 3. 总结历史
        ToolRetryMiddleware(max_retries=3),         # 4. 重试工具
    ]
)

# 高级 Middleware 模式

###  1、自定义状态（Custom State）

扩展 `AgentState` 添加自定义字段。

In [ ]:
from langchain.agents.middleware import AgentState, before_model
from typing_extensions import NotRequired
from langgraph.runtime import Runtime
from typing import Any

# 1. 定义自定义状态
class CustomState(AgentState):
    """扩展状态，添加自定义字段"""
    model_call_count: NotRequired[int]
    user_id: NotRequired[str]
    user_preferences: NotRequired[dict]

# 2. 使用自定义状态
@before_model(state_schema=CustomState, can_jump_to=["end"])
def check_user_quota(
    state: CustomState,
    runtime: Runtime
) -> dict[str, Any] | None:
    """检查用户配额"""

    user_id = state.get("user_id", "anonymous")
    call_count = state.get("model_call_count", 0)

    # 假设用户有配额限制
    user_quota = 100

    if call_count >= user_quota:
        print(f"⚠️  用户 {user_id} 已达配额上限")
        return {
            "jump_to": "end",
            "messages": [AIMessage(content="您的配额已用完，请升级套餐。")]
        }

    # 更新计数
    return {"model_call_count": call_count + 1}

# 3. 创建 Agent 时指定状态 schema
agent = create_agent(
    model=model,
    state_schema=CustomState,  # 👈 指定自定义状态
    middleware=[check_user_quota],
    tools=[]
)

# 4. 调用时传入自定义状态
result = agent.invoke({
    "messages": [{"role": "user", "content": "你好"}],
    "user_id": "user_123",
    "model_call_count": 95
})

### 2、动态模型选择

根据任务复杂度动态选择模型。

In [ ]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable
import os

class DynamicModelSelector:
    """动态模型选择器"""

    def __init__(self):
        # 初始化不同级别的模型
        self.simple_model = init_chat_model(
            "Qwen/Qwen2.5-7B-Instruct",
            model_provider="openai",
            base_url="https://api.siliconflow.cn/v1",
            api_key=os.getenv("SILICONFLOW_API_KEY")
        )

        self.complex_model = init_chat_model(
            "Qwen/Qwen2.5-72B-Instruct",
            model_provider="openai",
            base_url="https://api.siliconflow.cn/v1",
            api_key=os.getenv("SILICONFLOW_API_KEY")
        )

    def _estimate_complexity(self, messages: list) -> str:
        """估算查询复杂度"""
        if not messages:
            return "simple"

        last_message = str(messages[-1])

        # 简单规则判断
        complex_keywords = ["分析", "为什么", "如何", "原理", "详细", "复杂"]
        if any(kw in last_message for kw in complex_keywords):
            return "complex"

        # 消息长度判断
        if len(last_message) > 100:
            return "complex"

        return "simple"

    def create_hook(self):
        """创建动态选择钩子"""

        @wrap_model_call
        def select_model(
            request: ModelRequest,
            handler: Callable[[ModelRequest], ModelResponse]
        ) -> ModelResponse:
            # 估算复杂度
            complexity = self._estimate_complexity(request.messages)

            # 选择模型
            if complexity == "complex":
                print("🧠 使用大模型（复杂任务）")
                selected_model = self.complex_model
            else:
                print("⚡ 使用小模型（简单任务）")
                selected_model = self.simple_model

            # 修改请求的模型
            modified_request = request.override(model=selected_model)
            return handler(modified_request)

        return select_model

# 使用
selector = DynamicModelSelector()
agent = create_agent(
    model=selector.simple_model,  # 默认模型（会被动态替换）
    middleware=[selector.create_hook()],
    tools=[]
)

# 测试
queries = [
    "1+1等于多少？",  # 简单
    "请详细分析深度学习的反向传播原理"  # 复杂
]

for query in queries:
    print(f"\n{'='*60}")
    print(f"问题：{query}")
    result = agent.invoke({"messages": [{"role": "user", "content": query}]})
    print(f"回答：{result['messages'][-1].content[:100]}...")

### 3、成本追踪和预算控制

In [ ]:
from langchain.agents.middleware import AgentMiddleware, AgentState, ModelRequest, ModelResponse
from langgraph.runtime import Runtime
from typing import Any, Callable
from datetime import datetime

class CostTrackingMiddleware(AgentMiddleware):
    """成本追踪中间件"""

    def __init__(self, daily_budget: float = 10.0):
        self.daily_budget = daily_budget  # 每日预算（元）
        self.daily_cost = 0.0
        self.current_date = datetime.now().date()

        # 模型价格（元/百万 tokens）
        self.model_prices = {
            "Qwen/Qwen2.5-7B-Instruct": 0.35,
            "Qwen/Qwen2.5-72B-Instruct": 3.5,
            "openai:gpt-4o": 15.0,
            "openai:gpt-4o-mini": 1.5,
        }

    def _reset_if_new_day(self):
        """新的一天重置成本"""
        today = datetime.now().date()
        if today != self.current_date:
            print(f"📅 新的一天，重置成本统计")
            self.daily_cost = 0.0
            self.current_date = today

    def _estimate_cost(
        self,
        model_name: str,
        input_tokens: int,
        output_tokens: int
    ) -> float:
        """估算成本"""
        price_per_million = self.model_prices.get(model_name, 1.0)
        total_tokens = input_tokens + output_tokens
        cost = (total_tokens / 1_000_000) * price_per_million
        return cost

    def before_model(
        self,
        state: AgentState,
        runtime: Runtime
    ) -> dict[str, Any] | None:
        """检查预算"""
        self._reset_if_new_day()

        if self.daily_cost >= self.daily_budget:
            print(f"⚠️  已达每日预算上限 ¥{self.daily_budget}")
            return {
                "jump_to": "end",
                "messages": [AIMessage(content="今日预算已用完，请明天再试。")]
            }

        return None

    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse]
    ) -> ModelResponse:
        """追踪成本"""
        response = handler(request)

        # 提取 token 使用情况
        if hasattr(response, "usage_metadata"):
            usage = response.usage_metadata
            input_tokens = usage.get("input_tokens", 0)
            output_tokens = usage.get("output_tokens", 0)

            # 计算成本
            model_name = str(request.model)
            cost = self._estimate_cost(model_name, input_tokens, output_tokens)
            self.daily_cost += cost

            print(f"💰 本次成本：¥{cost:.4f}（累计：¥{self.daily_cost:.4f}/¥{self.daily_budget}）")

        return response

# 使用
cost_tracker = CostTrackingMiddleware(daily_budget=5.0)

agent = create_agent(
    model=model,
    middleware=[cost_tracker],
    tools=[]
)

### 4、Anthropic Prompt Caching

利用 Anthropic 的提示词缓存功能，大幅降低成本（适用于 Claude 模型）。

In [ ]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain_anthropic import ChatAnthropic
from typing import Callable

@wrap_model_call
def enable_prompt_caching(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """
    为 Anthropic 模型启用 Prompt Caching

    工作原理：
    1. 在系统消息中添加缓存标记
    2. 后续相同的系统消息会复用缓存
    3. 可节省高达 90% 的输入 token 成本
    """

    # 只对 Anthropic 模型生效
    if isinstance(request.model, ChatAnthropic):
        # 修改系统消息，添加缓存标记
        if request.system_message:
            # 为系统消息的最后一个内容块添加缓存标记
            content_blocks = list(request.system_message.content_blocks)
            if content_blocks:
                content_blocks[-1]["cache_control"] = {"type": "ephemeral"}

                new_system_message = request.system_message.copy()
                new_system_message.content = content_blocks

                modified_request = request.override(system_message=new_system_message)
                return handler(modified_request)

    return handler(request)

# 使用
from langchain_anthropic import ChatAnthropic

anthropic_model = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    api_key=os.getenv("ANTHROPIC_API_KEY")
)

agent = create_agent(
    model=anthropic_model,
    middleware=[enable_prompt_caching],
    tools=[]
)

# 结构化输出与数据验证

### 1、 使用 Pydantic 定义输出结构

结合 Middleware 和结构化输出，实现强类型的 Agent 响应。

In [ ]:
"""
结构化输出示例
"""

from pydantic import BaseModel, Field, field_validator
from typing import List, Optional
from datetime import datetime
from langchain.agents import create_agent
from langchain.agents.middleware import after_model, AgentState
from langchain.chat_models import init_chat_model
from langgraph.runtime import Runtime
from typing import Any
import os

# 1. 定义数据模型
class Task(BaseModel):
    """任务"""
    title: str = Field(description="任务标题")
    priority: int = Field(description="优先级 1-5", ge=1, le=5)
    deadline: Optional[str] = Field(description="截止日期 YYYY-MM-DD")
    tags: List[str] = Field(description="标签列表", default_factory=list)

    @field_validator('deadline')
    @classmethod
    def validate_deadline(cls, v):
        """验证日期格式"""
        if v:
            try:
                datetime.strptime(v, '%Y-%m-%d')
            except ValueError:
                raise ValueError('日期格式必须是 YYYY-MM-DD')
        return v

class ProjectPlan(BaseModel):
    """项目计划"""
    project_name: str = Field(description="项目名称")
    description: str = Field(description="项目描述")
    tasks: List[Task] = Field(description="任务列表")
    total_days: int = Field(description="预计总天数", gt=0)

    @field_validator('tasks')
    @classmethod
    def validate_tasks(cls, v):
        """验证至少有一个任务"""
        if len(v) == 0:
            raise ValueError('至少需要一个任务')
        return v

# 2. 创建模型
model = init_chat_model(
    "Qwen/Qwen3-8B",
    model_provider="openai",
    base_url="https://api.siliconflow.cn/v1",
    api_key= API_KEY
)

# 3. 使用结构化输出
structured_model = model.with_structured_output(ProjectPlan)

# 4. 添加验证中间件
@after_model
def validate_output(
    state: AgentState,
    runtime: Runtime
) -> dict[str, Any] | None:
    """验证输出格式"""
    last_message = state["messages"][-1]
    print(f"✅ 验证通过：生成了 {len(last_message.content.tasks)} 个任务")
    return None

# 5. 生成项目计划
prompt = """请为"开发一个知识库问答系统"项目制定计划。

要求：
1. 列出 4-6 个主要任务
2. 设置合理的优先级（1-5）
3. 估算截止日期
4. 添加相关标签
"""

result = structured_model.invoke(prompt)

# 6. 输出结果
print(f"\n{'='*60}")
print(f"项目：{result.project_name}")
print(f"描述：{result.description}")
print(f"预计工期：{result.total_days} 天")
print(f"\n任务列表：")

for i, task in enumerate(result.tasks, 1):
    print(f"\n{i}. {task.title}")
    print(f"   优先级：{'⭐' * task.priority}")
    print(f"   截止日期：{task.deadline or '未设置'}")
    print(f"   标签：{', '.join(task.tags)}")

## 2、带重试的结构化输出

当输出验证失败时自动重试。

In [ ]:
from pydantic import ValidationError
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

class StructuredOutputWithRetry:
    """带重试的结构化输出"""

    def __init__(self, schema: type[BaseModel], max_retries: int = 3):
        self.schema = schema
        self.max_retries = max_retries

    def create_hook(self):
        @wrap_model_call
        def retry_on_validation_error(
            request: ModelRequest,
            handler: Callable[[ModelRequest], ModelResponse]
        ) -> ModelResponse:
            """验证失败时重试"""

            for attempt in range(self.max_retries):
                try:
                    response = handler(request)

                    # 尝试验证
                    if hasattr(response, 'content') and isinstance(response.content, dict):
                        self.schema(**response.content)  # 验证
                        print(f"✅ 验证通过（尝试 {attempt + 1}）")

                    return response

                except ValidationError as e:
                    print(f"⚠️  验证失败（尝试 {attempt + 1}/{self.max_retries}）")
                    print(f"   错误：{e}")

                    if attempt < self.max_retries - 1:
                        # 在提示词中添加错误信息
                        error_msg = f"\n\n注意：上次输出格式有误：{e}\n请严格按照要求输出。"

                        # 修改系统消息
                        new_system = request.system_message.content + error_msg if request.system_message else error_msg
                        modified_request = request.override(
                            system_message=SystemMessage(content=new_system)
                        )
                        request = modified_request
                    else:
                        raise

            return response

        return retry_on_validation_error

# 使用
retry_handler = StructuredOutputWithRetry(schema=ProjectPlan, max_retries=3)

agent = create_agent(
    model=structured_model,
    middleware=[retry_handler.create_hook()],
    tools=[]
)



# 实战项目 - 生产级智能研究助手

###  项目需求

构建一个生产级的智能研究助手，具备：

1. ✅ **多步推理** - 分解复杂问题
2. ✅ **信息搜索** - 搜索相关资料
3. ✅ **性能监控** - 追踪耗时和成本
4. ✅ **成本优化** - 动态模型选择
5. ✅ **错误恢复** - 自动重试和降级
6. ✅ **结构化输出** - 生成标准化报告
7. ✅ **敏感信息保护** - 检测和处理 PII

In [8]:
"""
实战项目：生产级智能研究助手（LangChain 1.0）

使用官方 Middleware API，包含：
- 性能监控
- 成本追踪
- 错误重试
- 结构化输出
- 动态模型选择
"""


from langchain.agents import create_agent
from langchain.agents.middleware import (
    AgentMiddleware,
    AgentState,
    ModelRequest,
    ModelResponse,
    before_model,
    after_model,
    wrap_model_call,
    wrap_tool_call,
    SummarizationMiddleware,
    ModelCallLimitMiddleware,
)
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, SystemMessage
from langchain.tools.tool_node import ToolCallRequest
from langchain_core.messages import ToolMessage
from langgraph.types import Command
from langgraph.runtime import Runtime
from pydantic import BaseModel, Field
from typing import List, Optional, Any, Callable
from datetime import datetime
import json
import time


# ==================== 数据模型 ====================

class SubQuestion(BaseModel):
    """子问题"""
    question: str = Field(description="子问题内容")
    priority: int = Field(description="优先级1-5", ge=1, le=5)
    difficulty: str = Field(description="难度：easy/medium/hard")

class ResearchPlan(BaseModel):
    """研究计划"""
    main_question: str = Field(description="主问题")
    sub_questions: List[SubQuestion] = Field(description="子问题列表")
    approach: str = Field(description="研究方法")
    estimated_time: str = Field(description="预计耗时")

class ResearchFinding(BaseModel):
    """研究发现"""
    topic: str = Field(description="主题")
    key_points: List[str] = Field(description="关键要点")
    sources: List[str] = Field(description="信息来源")
    confidence: float = Field(description="置信度0-1", ge=0, le=1)

class ResearchReport(BaseModel):
    """研究报告"""
    title: str = Field(description="报告标题")
    summary: str = Field(description="执行摘要")
    findings: List[ResearchFinding] = Field(description="研究发现")
    conclusion: str = Field(description="结论")
    recommendations: List[str] = Field(description="建议")
    generated_at: str = Field(description="生成时间")

# ==================== 工具定义 ====================

@tool
def search_information(query: str) -> str:
    """搜索信息（模拟知识库）

    Args:
        query: 搜索关键词
    """
    knowledge = {
        "LangChain": "LangChain 1.0 是一个构建 LLM 应用的框架，核心特性包括 Middleware、结构化输出、Agent 等。",
        "Middleware": "Middleware 提供钩子系统，包括 before_model、after_model、wrap_model_call 等，用于控制 Agent 执行流程。",
        "RAG": "RAG（检索增强生成）结合信息检索和文本生成，通过向量数据库检索相关文档，提高回答准确性。",
        "Agent": "Agent 是能够使用工具、进行推理的 AI 系统，LangChain 1.0 提供了强大的 Agent 框架。",
    }

    results = []
    for key, value in knowledge.items():
        if query.lower() in key.lower() or key.lower() in query.lower():
            results.append(f"【{key}】{value}")

    return "\n\n".join(results) if results else f"未找到关于'{query}'的信息"

@tool
def analyze_data(data: str) -> str:
    """分析数据

    Args:
        data: 需要分析的数据
    """
    word_count = len(data.split())
    char_count = len(data)

    return f"""数据分析结果：
- 总字数：{word_count}
- 总字符数：{char_count}
- 信息完整性：{'高' if word_count > 50 else '中' if word_count > 20 else '低'}
- 包含关键概念数：{len([k for k in ["LangChain", "Middleware", "RAG", "Agent"] if k in data])}
"""

# ==================== Middleware 定义 ====================

class PerformanceMonitorMiddleware(AgentMiddleware):
    """性能监控中间件"""

    def __init__(self):
        self.session_start = None
        self.model_calls = []
        self.tool_calls = []

    def before_agent(
        self,
        state: AgentState,
        runtime: Runtime
    ) -> dict[str, Any] | None:
        self.session_start = time.time()
        print(f"\n{'='*70}")
        print(f"🚀 会话开始：{datetime.now().strftime('%H:%M:%S')}")
        print(f"{'='*70}\n")
        return None

    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse]
    ) -> ModelResponse:
        start = time.time()
        try:
            response = handler(request)
            elapsed = time.time() - start
            self.model_calls.append(elapsed)
            print(f"⏱️  模型调用 #{len(self.model_calls)}：{elapsed:.2f}秒")
            return response
        except Exception as e:
            print(f"❌ 模型调用失败：{e}")
            raise

    def wrap_tool_call(
        self,
        request: ToolCallRequest,
        handler: Callable[[ToolCallRequest], ToolMessage | Command]
    ) -> ToolMessage | Command:
        start = time.time()
        tool_name = request.tool_call["name"]
        try:
            result = handler(request)
            elapsed = time.time() - start
            self.tool_calls.append((tool_name, elapsed))
            print(f"🔧 工具调用 ({tool_name})：{elapsed:.2f}秒")
            return result
        except Exception as e:
            print(f"❌ 工具调用失败 ({tool_name})：{e}")
            raise

    def after_agent(
        self,
        state: AgentState,
        runtime: Runtime
    ) -> dict[str, Any] | None:
        total_time = time.time() - self.session_start

        print(f"\n{'='*70}")
        print(f"📊 性能统计")
        print(f"{'='*70}")
        print(f"总耗时：{total_time:.2f}秒")
        print(f"模型调用：{len(self.model_calls)} 次")
        if self.model_calls:
            print(f"  - 平均：{sum(self.model_calls)/len(self.model_calls):.2f}秒")
            print(f"  - 总计：{sum(self.model_calls):.2f}秒")
        print(f"工具调用：{len(self.tool_calls)} 次")
        if self.tool_calls:
            total_tool_time = sum(t[1] for t in self.tool_calls)
            print(f"  - 总计：{total_tool_time:.2f}秒")
        print(f"{'='*70}\n")

        return None

class CostTrackingMiddleware(AgentMiddleware):
    """成本追踪中间件"""

    def __init__(self, budget: float = 1.0):
        self.budget = budget
        self.total_cost = 0.0
        self.model_prices = {
            "Qwen/Qwen3-8B": 0.5,
            "Qwen/Qwen2.5-7B-Instruct": 0.3,
        }

    def before_model(
        self,
        state: AgentState,
        runtime: Runtime
    ) -> dict[str, Any] | None:
        if self.total_cost >= self.budget:
            print(f"⚠️  预算已用完（¥{self.budget}）")
            return {
                "jump_to": "end",
                "messages": [AIMessage(content="预算已用完")]
            }
        return None

    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse]
    ) -> ModelResponse:
        response = handler(request)

        # 估算成本（简化版）
        model_name = "Qwen/Qwen2.5-7B-Instruct"
        price = self.model_prices.get(model_name, 0.3)

        # 估算 tokens（简化）
        input_tokens = sum(len(str(m.content)) for m in request.messages) * 1.5
        print(response)
        output_tokens = len(str(response.result[0].content)) * 1.5

        cost = (input_tokens + output_tokens) / 1_000_000 * price
        self.total_cost += cost

        print(f"💰 成本：¥{cost:.4f}（累计：¥{self.total_cost:.4f}/¥{self.budget}）")

        return response

class RetryMiddleware(AgentMiddleware):
    """重试中间件"""

    def __init__(self, max_retries: int = 2):
        self.max_retries = max_retries

    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse]
    ) -> ModelResponse:
        for attempt in range(self.max_retries + 1):
            try:
                return handler(request)
            except Exception as e:
                if attempt == self.max_retries:
                    print(f"❌ 重试{self.max_retries}次后仍失败：{e}")
                    raise

                wait_time = 2 ** attempt
                print(f"🔄 重试中（{attempt + 1}/{self.max_retries}），{wait_time}秒后重试")
                time.sleep(wait_time)

# ==================== 智能研究助手 ====================

class ResearchAssistant:
    """智能研究助手"""

    def __init__(self):
        # 创建模型
        self.model = init_chat_model(
            "Qwen/Qwen3-8B",
            model_provider="openai",
            base_url="https://api.siliconflow.cn/v1",
            api_key= API_KEY,
            temperature=0.3
        )

        # 工具
        self.tools = [search_information, analyze_data]

        # Middleware
        self.performance_monitor = PerformanceMonitorMiddleware()
        self.cost_tracker = CostTrackingMiddleware(budget=0.5)
        self.retry_handler = RetryMiddleware(max_retries=2)

        # 创建 Agent
        self.agent = create_agent(
            model=self.model,
            tools=self.tools,
            middleware=[
                self.cost_tracker,           # 1. 成本控制
                ModelCallLimitMiddleware(run_limit=15),  # 2. 调用限制
                self.retry_handler,          # 3. 重试
                self.performance_monitor,    # 4. 性能监控
            ],
            system_prompt="""你是专业的研究助手。

            职责：
            1. 分解复杂问题为子问题
            2. 使用工具搜索和分析信息
            3. 生成结构化的研究报告

            工作流程：
            1. 理解研究主题
            2. 使用 search_information 搜索相关信息
            3. 使用 analyze_data 分析数据
            4. 综合信息生成结论
            """
        )

    def create_plan(self, question: str) -> ResearchPlan:
        """制定研究计划"""
        print(f"📋 正在制定研究计划...\n")

        structured_model = self.model.with_structured_output(ResearchPlan)

        prompt = f"""请为以下研究问题制定详细计划：

        研究问题：{question}

        要求：
        1. 分解为 3-4 个子问题
        2. 设置优先级（1-5）
        3. 评估难度（easy/medium/hard）
        4. 说明研究方法
        5. 估算所需时间
        """

        plan = structured_model.invoke(prompt)
        return plan

    def research(self, sub_question: str) -> str:
        """研究单个子问题"""
        print(f"\n🔍 研究子问题：{sub_question}\n")

        result = self.agent.invoke({
            "messages": [{
                "role": "user",
                "content": f"请搜索并分析关于'{sub_question}'的信息"
            }]
        })

        return result["messages"][-1].content

    def generate_report(
        self,
        question: str,
        findings: List[str]
    ) -> ResearchReport:
        """生成研究报告"""
        print(f"\n📝 正在生成研究报告...\n")

        structured_model = self.model.with_structured_output(ResearchReport)

        findings_text = "\n\n".join([
            f"发现 {i+1}：\n{f}"
            for i, f in enumerate(findings)
        ])

        prompt = f"""基于以下研究发现，生成完整的研究报告：

        研究问题：{question}

        研究发现：
        {findings_text}

        要求：
        1. 撰写执行摘要
        2. 整理关键发现（每个发现包含主题、要点、来源、置信度）
        3. 得出结论
        4. 提供建议
        """

        report = structured_model.invoke(prompt)
        report.generated_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        return report

    def conduct_research(self, question: str) -> ResearchReport:
        """执行完整研究流程"""
        print(f"\n{'='*70}")
        print(f"🎯 开始研究")
        print(f"{'='*70}")
        print(f"\n主题：{question}\n")

        # 1. 制定计划
        plan = self.create_plan(question)

        print(f"\n✅ 计划制定完成")
        print(f"研究方法：{plan.approach}")
        print(f"预计耗时：{plan.estimated_time}")
        print(f"\n子问题：")
        for i, sq in enumerate(plan.sub_questions, 1):
            print(f"{i}. [{sq.difficulty.upper()}] {sq.question}")

        # 2. 逐个研究
        findings = []
        for i, sq in enumerate(plan.sub_questions, 1):
            print(f"\n{'='*70}")
            print(f"进度：{i}/{len(plan.sub_questions)}")
            print(f"{'='*70}")

            finding = self.research(sq.question)
            findings.append(finding)

        # 3. 生成报告
        report = self.generate_report(question, findings)

        return report

def main():
    """主程序"""
    print(f"\n{'='*70}")
    print(f"🔬 智能研究助手（LangChain 1.0 Middleware）")
    print(f"{'='*70}\n")

    assistant = ResearchAssistant()

    # 研究主题
    topic = "LangChain 1.0 Middleware 系统的核心特性和应用场景"

    # 执行研究
    report = assistant.conduct_research(topic)

    # 输出报告
    print(f"\n{'='*70}")
    print(f"📊 研究报告")
    print(f"{'='*70}\n")
    print(f"标题：{report.title}")
    print(f"\n摘要：\n{report.summary}")
    print(f"\n研究发现：")
    for i, finding in enumerate(report.findings, 1):
        print(f"\n{i}. {finding.topic}")
        print(f"   置信度：{finding.confidence:.0%}")
        for point in finding.key_points:
            print(f"   • {point}")

    print(f"\n结论：\n{report.conclusion}")

    print(f"\n建议：")
    for i, rec in enumerate(report.recommendations, 1):
        print(f"{i}. {rec}")

    print(f"\n生成时间：{report.generated_at}")

    # 保存报告
    filename = f"research_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(report.model_dump(), f, ensure_ascii=False, indent=2)

    print(f"\n✅ 报告已保存：{filename}\n")

if __name__ == "__main__":
    main()


🔬 智能研究助手（LangChain 1.0 Middleware）


🎯 开始研究

主题：LangChain 1.0 Middleware 系统的核心特性和应用场景

📋 正在制定研究计划...


✅ 计划制定完成
研究方法：为研究问题 'LangChain 1.0 Middleware 系统的核心特性和应用场景' 制定详细研究计划，分解为 3-4 个子问题，并设置优先级、难度评估、研究方法和时间估算。
预计耗时：约 10-15 天

子问题：
1. [MEDIUM] LangChain 1.0 Middleware 的核心架构和功能模块是什么？
2. [MEDIUM] LangChain 1.0 Middleware 支持哪些类型的集成和扩展？
3. [HARD] LangChain 1.0 Middleware 在实际应用场景中有哪些典型用例？
4. [MEDIUM] LangChain 1.0 Middleware 的性能、安全性和可扩展性如何？

进度：1/4

🔍 研究子问题：LangChain 1.0 Middleware 的核心架构和功能模块是什么？


🚀 会话开始：13:29:36

⏱️  模型调用 #1：1.56秒
ModelResponse(result=[AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 357, 'total_tokens': 390, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '019b0873f8c0

LengthFinishReasonError: Could not parse response content as the length limit was reached - CompletionUsage(completion_tokens=4096, prompt_tokens=663, total_tokens=4759, completion_tokens_details=None, prompt_tokens_details=None)